# Module 4: Data Sharing and Next Steps.



As you have perfomed before:

1.) Click on File on the top left corner and select save a copy in drive
Your changes will not be saved if you do not do this step.

2.) Click on the name of the workbook in the top left corner and replace "Copy of" with your full name.

### Let's install analysis packages needed.

For this portion we will use:  
**seqtk** for assess genome quality (https://github.com/lh3/seqtk)  
**Mafft** for genome alignment (https://mafft.cbrc.jp/alignment/software/)  
**snp-site** for quick SNP difference assessment (https://github.com/sanger-pathogens/snp-sites)  
**Fasttree** for phylogenetic tree building (http://www.microbesonline.org/fasttree/)  
**Phylo** from biopython for quick tree visualisation (https://biopython.org/wiki/Phylo). *Note*: there are lots of tree visualisaiton programmes, most commonly used are ggtree(R), ete3 (python) and itol (https://itol.embl.de/).  

microreact account (https://microreact.org/), which you can setup/sign in with your google account.

In [ ]:
!python --version

Python 3.7.15


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c conda-forge biopython

In [ ]:
!conda install -c bioconda mafft snp-sites fasttree seqtk snp-dists


### Download dataset to be analysed

In this module we will use the data analyzed in module 3. This data belongs to the article "[Overview of the SARS-CoV-2 genotypes circulating in Latin America during 2021]( https://doi.org/10.3389/fpubh.2023.1095202 )" published by Molina-Mora et al. 2023, as part of a CABANA project.

In [ ]:
!wget https://zenodo.org/records/10888461/files/module_4.tar

In [ ]:
!tar -xvf module_4.tar

In [ ]:
%cd module_4/

First we will assess genome quality to see if the genomes are good enough for analysis. For SARS-CoV-2, we are mainly interested in the number of Ns. The package we will use is seqtk comp, which gives a lots of statistics for a given sequence file in fasta format.   

Output format: chr, length, #A, #C, #G, #T, #2, #3, #4, #CpG, #tv, #ts, #CpG-ts

9th column, #4 is the number of Ns, 4 ambiguities, ie A, G, T, or C.

If you have separated fasta files, it might be easier to combine the files together.  Otherwise, you have to go through the genome files separately.

In [ ]:
#seqtk comp analysis the sequence information
#cut -f 1,9 selects the column 1 and 9, the information we need.
!seqtk comp gisaid_hcov-19_2024_03_27_02.fasta    | cut -f 1,9 > gisaid_hcov-19_quality.tsv

In [ ]:
!cat gisaid_hcov-19_quality.tsv

The quality looks good, however, samples EPI_ISL_10072006 and EPI_ISL_7961355 are not as good as the others. They have 1139 and 1628 ambiguous bases, but they are good enough for analysis. Overall, we require 90% or less than **3000** Ns in the genome for assembly and phylogenetic tree analysis. Having 70% of the genome assembled is the default value in Pangolin for lineage assignment.

### Alignment

We will use mafft for alignment. It is fairly fast and pretty accurate.

There are many options for alignning sequences in mafft. `--auto` is a good option where the programme itself chooses the most efficient (good balance between speed and accuracy) algorithm. Alignment could take days to align long sequences if using the most accurate algorthm.If you are aligning short sequences, such as a region of the spike gene/protein, you can use more accurate options. The mafft website has good examples of what to use when.

For mafft to align the sequences, in our case whole genomes, you need to combine the sequence you want to align into one file. Good thing is you have done that already and since all genomes are good we don't need to exclude any from the alignment. 

In [ ]:
!mafft --auto gisaid_hcov-19_2024_03_27_02.fasta > gisaid_hcov_aligned.fasta

### Quick look at the genome differences
snp-site is a really good software to give you an idea how closely related your genomes of interest are. For small datasets such as this, this is really good. It analyses your alignment and give you a SNP alignmern as default. it can also give you snp information in VCF format.

In [ ]:
!snp-sites gisaid_hcov_aligned.fasta

In [ ]:
#You can use the snp-dists command to calculate the pairwise SNP distances between the sequences in the alignment.
!snp-dists gisaid_hcov_aligned.fasta

The alignment is a little bit more complicated with more differences, we need a phylogenetic tree to visualise the relationships

In [ ]:
!fasttree -nt -gtr -gamma gisaid_hcov_aligned.fasta > gisaid_hcov_aligned.nwk

In [ ]:
!cat gisaid_hcov_aligned.nwk
#as you can see it is quite difficult to interpret this format without visualising in tree-form


In [ ]:
from Bio import Phylo

tree = Phylo.read("gisaid_hcov_aligned.nwk", "newick")
print("tree")
Phylo.draw_ascii(tree)

In [ ]:
#let's add the 2020 reference genome to the dataset. 
!mafft --auto --add gisaid_hcov_aligned.fasta reference_genome.fasta > gisaid_hcov_aligned_ref.fasta

In [ ]:
!fasttree -nt -gtr -gamma gisaid_hcov_aligned_ref.fasta > gisaid_hcov_aligned_ref.nwk

Have a look at the snp alignment as well using snp-site. You will see that using the reference allowed the identification of ancestral bases, so now we know what nucleotide changes are mutations from wildtype. Previously the tree algorithm made the assumption that the majority base is the ancestral sequence, which is not correct.

In [ ]:
from Bio import Phylo

treeOutgroup = Phylo.read("gisaid_hcov_aligned_ref.nwk", "newick")
treeOutgroup.root_with_outgroup({"name": "nCoV2019|Wuhan-Hu-1|MN908947|China|Wuhan|2019-12"}) 
print("treeOutgroup")
Phylo.draw_ascii(treeOutgroup)

### watch microreact and nextstrain demo/video

looking at trees this way is pretty difficult, there are some good tools out there that allows interactive trees and metadate visualisation.

https://docs.microreact.org/   
https://nextstrain.org/community/narratives/ESR-NZ/GenomicsNarrativeSARSCoV2/aotearoa-border-incursions

### Other tools for larger datasets and tree building
mafft even at the minimum accuracy model can be quite slow once you have 1000s of genomes to align. To speed things up, you can add new genomes to existing alignments or divide genomes into groups such as lineages or sublineages and then merge the alignments. The mafft website has good explaination of these tricks and their issues.    

nextalign is another tool you can use. It is super fast! (https://docs.nextstrain.org/projects/nextclade/en/stable/user/nextalign-cli.html)  

iqtree2 is a comprehensive (not Baysian) phylogeny building package that can test for best substition models for your dataset, perform bootstrap analysis, compare phylogeny etc etc. It is probably one of the best tree building tools (that is reasnabley fast) out there currently. (http://www.iqtree.org/)

usher is a software package that allows adding sequences to existing trees without doing all the alignment again. (https://www.nature.com/articles/s41588-021-00862-7) We used this when we analysed the flight dataset. There are some caveats and issues you need to be aware of. If the existing alignment is really clean and the sequences to be added are high-quality, using usher is a good fast way to get some quick preliminary results. However, it does not deal with Ns or gaps very well.  

For timed baysian phyolgeny, BEAST2 is often used. (https://www.beast2.org/)  https://www.nature.com/articles/s41559-017-0280-x  

## Assignment

> **There is a clade containing the 3 sequences from a single country. What country is it?** 

> **What is the distance between EPI_ISL_14434222 and EPI_ISL_14434358?**

> **How is it reflected in the tree?**